<a href="https://colab.research.google.com/github/MerkulovDaniil/optim/blob/master/assets/Notebooks/LLM_fine_tuning_on_songs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎤 Давайте дообучим ruGPT3 на текстах песен

В этом практикуме мы увидим, как можно дообучать современные языковые модели на текстах песен.

<img src="https://raw.githubusercontent.com/oseledets/dl2023/main/seminars/seminar-10/neuro_kish.jpg" alt= “” width="128px">

## 👨‍🍳 Пререквизиты

In [1]:
# Загрузим датасеты
!wget -N https://github.com/MerkulovDaniil/mipt21/raw/main/assets/lyrics.zip
!unzip -q -o lyrics.zip -d ./

!pip install -q sentencepiece
!pip install -q transformers  datasets
!pip install -q accelerate
# !pip install -q deepspeed mpi4py
!pip install -q pynvml
!pip install -q wandb

import torch
import pandas as pd
import time

class Profiler():

    def __init__(self,) -> None:
        pass

    def gpu_mem(self):
        mem = torch.cuda.mem_get_info()
        mb = list(map(lambda x:x/pow(2,20),mem))
        total = mb[1]
        used = mb[1]-mb[0]
        return used,total

    def gpu_mem_info(self,title = ''):
        used,total = self.gpu_mem()
        print(f'🤖 {title} gpu mem : {used:.1f}/{total:.1f} mb')

    def one_step_report(self,batch, model, optimizer, do_backward = True,device = torch.device('cpu'),print_loss = False,deepspeed = False):

        report_df = pd.DataFrame(columns=['used_mem','delta_mem','delta_time'])

        delta_time =[0]
        used_mem = [self.gpu_mem()[0]]

        self.gpu_mem_info('begin')

        model.train()

        ids = batch['input_ids'].to(device,dtype=torch.long)
        labels = batch['labels'].to(device,dtype=torch.long)

        torch.cuda.synchronize()
        start_time = time.time()

        outputs = model(input_ids = ids,labels = labels)
        loss = outputs[0]

        torch.cuda.synchronize()
        forward_time = time.time()
        delta_time.append(-start_time + forward_time)

        used_mem.append(self.gpu_mem()[0])
        self.gpu_mem_info(f'{delta_time[-1]:.3f}s forward')
        if do_backward:
            optimizer.zero_grad()
            if deepspeed:
                model.backward(loss)
            else:
                loss.backward()

            torch.cuda.synchronize()
            backward_time = time.time()
            delta_time.append(-forward_time + backward_time)
            used_mem.append( self.gpu_mem()[0])
            self.gpu_mem_info(f'{delta_time[-1]:.3f}s backward')

            if deepspeed:
                model.step()
            else:
                optimizer.step()

            torch.cuda.synchronize()
            optimizer_step_time = time.time()
            delta_time.append(-backward_time + optimizer_step_time)
            used_mem.append( self.gpu_mem()[0])
            self.gpu_mem_info(f'{delta_time[-1]:.3f}s optimizer_step')

        if (print_loss):
            print('loss',loss)

        torch.cuda.empty_cache()
        used_mem.append( self.gpu_mem()[0])
        torch.cuda.synchronize()
        end_time = time.time()
        delta_time.append(end_time - optimizer_step_time)
        #
        report_df.loc[:,'used_mem'] = pd.Series(used_mem)
        report_df.loc[:,'delta_time'] = pd.Series(delta_time)
        indexes = ['begin','forward','backward','optim_step','end']
        report_df.index = indexes

        report_df['delta_mem'] =  report_df['used_mem']- report_df.loc['begin','used_mem']

        report_df.loc['total'] = [self.gpu_mem()[1],0,end_time-start_time]
        report_df['delta_time'] = report_df['delta_time'].map(lambda t : round(t,3))

        return report_df

prof = Profiler()

prof.gpu_mem()
# (5804.0, 15109.75)

prof.gpu_mem_info()
# gpu mem : 5840.0/15109.8 mb


# report = prof.one_step_report(batch, model,optim,device = DEVICE)
# # begin gpu mem : 5804.0/15109.8 mb
# # 0.050s forward gpu mem : 13006.0/15109.8 mb
# # 1.232s backward gpu mem : 14576.0/15109.8 mb
# # 0.025s optimizer_step gpu mem : 14576.0/15109.8 mb

# report

# Simplest memory profiling
def gpu_mem():
    mem = torch.cuda.mem_get_info()
    mb = list(map(lambda x:x/pow(2,20),mem))
    total = mb[1]
    used = mb[1]-mb[0]
    return used,total

def gpu_mem_info(title = ''):
    used,total = gpu_mem()
    print(f'🤖 {title} gpu mem : {used:.1f}/{total:.1f} mb')

--2024-06-25 12:50:11--  https://github.com/MerkulovDaniil/mipt21/raw/main/assets/lyrics.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MerkulovDaniil/mipt21/main/assets/lyrics.zip [following]
--2024-06-25 12:50:11--  https://raw.githubusercontent.com/MerkulovDaniil/mipt21/main/assets/lyrics.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1062172 (1.0M) [application/zip]
Saving to: ‘lyrics.zip’

lyrics.zip          100%[===================>]   1.01M  --.-KB/s    in 0.008s  

Last-modified header missing -- time-stamps turned off.
2024-06-25 12:50:12 (123 MB/s) - ‘lyrics.zip’ saved [

## 📦 Выберем датасет и модель

In [3]:
MODEL_NAME = 'ai-forever/rugpt3small_based_on_gpt2' #@param ['ai-forever/rugpt3small_based_on_gpt2', 'ai-forever/rugpt3medium_based_on_gpt2','ai-forever/rugpt3large_based_on_gpt2', 'gpt2-large']
DATASET_PATH = './kish_lyrics.txt'  #@param ['./manowar_lyrics.txt','./kish_lyrics.txt','./korzh_lyrics.txt', './oxxxy_lyrics.txt', './pushkin.txt', './Алла Пугачёва_lyrics.txt', './Ария_lyrics.txt', './Виа ГРА_lyrics.txt', './Владимир Высоцкий_lyrics.txt', './Дима Билан_lyrics.txt', './Заточка_lyrics.txt', './Кино_lyrics.txt', './Михаил Круг_lyrics.txt', './Скриптонит_lyrics.txt']

with open(DATASET_PATH) as text_file:
    text = text_file.read().splitlines()
    print(text)

['', 'Тёмный, мрачный коридор', 'Я на цыпочках, как вор', 'Пробираюсь, чуть дыша', 'Чтобы не спугнуть', 'Тех, кто спит уже давно', 'Тех, кому не всё равно', 'В чью я комнату тайком', 'Желаю заглянуть', 'Чтобы увидеть...', '', 'Как бессонница в час ночной', 'Меняет, нелюдимая, облик твой', 'Чьих, невольница, ты идей?', 'Зачем тебе охотиться на людей?', '', 'Крестик на моей груди', 'На него ты погляди', 'Что в тебе способен он', 'Резко изменить?', 'Много книжек я читал', 'Много фокусов видал', 'Свою тайну от меня', 'Не пытайся скрыть!', 'Я это видел!', 'Как бессонница в час ночной', 'Меняет, нелюдимая, облик твой', 'Чьих, невольница, ты идей?', 'Зачем тебе охотиться на людей?', '', 'Очень жаль, что ты тогда', 'Мне поверить не смогла', 'В то, что новый твой приятель', 'Не такой, как все!', 'Ты осталась с ним вдвоём', 'Не зная ничего о нём', 'Что для всех опасен он', 'Наплевать тебе!', 'И ты попала!', '', 'К настоящему колдуну', 'Он загубил таких, как ты, не одну!', 'Словно куклой, в час н

## 💃 Model loading

In [4]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import transformers
from sklearn.model_selection import train_test_split
import time
import pandas as pd
import random
# import deepspeed

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"🤖 Working on {DEVICE}")

model_name_or_path = MODEL_NAME
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

🤖 Working on cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

In [5]:
print(f"🤖The total number of parameters in the model is {model.num_parameters()}")
gpu_mem_info()

🤖The total number of parameters in the model is 125231616
🤖  gpu mem : 673.0/15102.1 mb


## 🎁 Взгляд на данные

In [6]:
if 'pushkin' in DATASET_PATH or 'mayakovskiy' in DATASET_PATH:
    tokenizer.add_tokens('</s>')
    tokenizer.add_special_tokens({
        'eos_token': '</s>',
        'pad_token': '<pad>'
    })
if 'lyrics' in DATASET_PATH:
    tokenizer.add_tokens('[EOS]')
    tokenizer.add_special_tokens({
        'eos_token': '[EOS]',
        'pad_token': '<pad>'
    })

model.resize_token_embeddings(len(tokenizer))

# Dataset
train_dataset = TextDataset(tokenizer=tokenizer,file_path=DATASET_PATH,block_size=512)
train_dataset, eval_dataset = train_test_split(train_dataset,test_size = 0.1,random_state = 42)

# Creating a data_collator (slices the text into optimal length pieces)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [8]:
import numpy as np
print(f"🤖 Размер словаря (количество различных токенов): {tokenizer.vocab_size}")
print(f"🤖 Случайные примеры токенов: {[tokenizer.decode(id) for id in np.random.choice(range(tokenizer.vocab_size), 4)]}")

🤖 Размер словаря (количество различных токенов): 50257
🤖 Случайные примеры токенов: ['use', ' нехорошо', ' коричне', 'фести']


In [12]:
print(f"🤖 Как выглядит первый батч из обучающей выборки размера {len(train_dataset[0])}")
print(train_dataset[1])

🤖 Как выглядит первый батч из обучающей выборки размера 512
tensor([32702,  3087, 12786, 30381, 15627,    16,  2152,  1257,  1905,   300,
          203,   732,  6061,   749, 40958, 30618,   300,   203,   618,   785,
         2119, 15096,    16,   374,   294,  5414,   651,   203,  1140,  1013,
         8740,  1166,   975,     5,   203,   203,   203, 50257,   203,   203,
          203,   983,  3080,   305, 12419,  1152, 12809,   203,  1009,  1905,
          668,   519, 22873,   203,   677,  5258,   668, 19588,   203,   677,
         1851,    17,   299,  6885,  9792,   203,   732, 26460,   349,  6915,
          467,   203, 30266, 12396,   635, 38756,   203,   732,   282,  8160,
          547,   327, 44508,  1543,   467,   203, 10150,   280,  1133,  8113,
            5, 42065, 32702,  3087, 12786,   203,   203,   203, 50257,   203,
          203,   203,  8572,   736,  1157, 29202,   289,  3609,  1891,   203,
          672,   925,  1933,   877,   281,  5963,   519,  4079,   203,  3743,
    

In [13]:
# Testing
tokenizer.decode(train_dataset[1])

' might also likeВремя шло, народ оголодал\nИ Иван без крошки пропадал\nА когда смекнули, что к чему –\nУсы оторвали ему!\n\n\n[EOS]\n\n\nБросает клоун тень\nГолодную до смеха\nВ шумную толпу\nВ какой-то миг успеха\nИ тучи застилают\nНад цирком небеса\nИ в чёрном чреве тают\nЛюдские голоса!You might also like\n\n\n[EOS]\n\n\nРаботники ножа и топора\nПили брагу с ночи до утра\nНо какой-то гоблин выбежал из леса\nСел за стол и заявил:\nНикогда с крутыми я не пил!\n\nТвои слова ненужный хлам!\nМы будем лучше пить и е-е-есть!\nТвоя\u205fодежда\u205fсразу\u205fнам\nСказала, нам кто\u205fты есть! (нам\u205fкто ты есть!)\n\nЯ и мой народ живем в горах\nЯ парень - будь здоров в своих кругах!\nМне нельзя хамить и надо мной шутить\nСо мной нельзя играть!\nПоверьте, я, как вы, умею обижать!\n\nТвои слова ненужный хлам!\nМы будем лучше пить и е-е-есть!\nТвоя одежда сразу нам\nСказала, нам кто ты есть! (нам кто ты есть!)\n\nПослушай гоблин, ты много болтаешь!\nМолись своим богам!\nНо я же как один 

## 👨‍🏫 Обучение

In [14]:
batch_size = 6
n_epochs = 20
training_args = TrainingArguments(
    output_dir="./finetuned", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=n_epochs, # number of training epochs
    warmup_steps=10,# number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=4, # to make "virtual" batch size larger
    evaluation_strategy  = 'epoch',
    logging_strategy  = 'epoch',
    save_strategy  = 'epoch',
    auto_find_batch_size = True,
    load_best_model_at_end = True,
    report_to="none",
    )


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset = eval_dataset,
    # optimizers = (torch.optim.NAdam(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [16]:
trainer_log = trainer.train()

Epoch,Training Loss,Validation Loss
0,3.911500,3.273982
2,3.633400,3.157491
4,3.368000,3.095879
6,3.111600,3.067601
8,2.941800,3.056233


Epoch,Training Loss,Validation Loss
0,3.911500,3.273982
2,3.633400,3.157491
4,3.368000,3.095879
6,3.111600,3.067601
8,2.941800,3.056233
10,2.789500,3.058142
12,2.704400,3.058290
14,2.647400,3.054997
16,2.615500,3.059375
17,1.116900,3.059391


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


## 🗣️ Генерация

In [23]:
import torch

# Примерный текст для начала генерации
text = "В старом замке, стоящем на краю деревни, местные жители часто видели призрачные огни. В эту ночь юный Иван решил исследовать это место и, зайдя внутрь, увидел"

# Подготовка входных данных для модели
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
attention_mask = torch.ones_like(input_ids)  # Все значения изначально устанавливаются в 1
pad_token_id = 50256  # EOS token ID

# Установим attention mask и pad token ID
if pad_token_id is not None:
    attention_mask[input_ids == pad_token_id] = 0  # Установим attention mask в 0 для pad токенов

# Генерация текста с использованием модели
model.eval()
with torch.no_grad():
    out = model.generate(input_ids,
                         do_sample=True,
                         num_beams=4,
                         temperature=2.0,
                         top_k=50,
                         top_p=0.9,
                         repetition_penalty=2.0,
                         max_length=200,
                         attention_mask=attention_mask
                         )

generated_text = list(map(tokenizer.decode, out))[0]
print("\nGenerated Text:\n")
print(generated_text)


Generated Text:

В старом замке, стоящем на краю деревни, местные жители часто видели призрачные огни. В эту ночь юный Иван решил исследовать это место и, зайдя внутрь, увидел жутких гоблинов, рыскающих у его ног

Иван с ружьём в руке стал спускаться по крутой деревянной лестнице
Вдруг откуда ни возьмись появилась огромная тень
Участок за участком светлели дома
Гоблины крались по пятам за юношей
«Не бойся, парень, мы не привидения!» — шептали они
Но юноша ничего не замечал
Он всё глубже погружался в темноту
И вдруг раздался зловещий крик!

Охотник закричал: «Хой! Это я!»
Крикнул он и в тот же миг грохнул выстрел!
Лишь спустя несколько минут охотник смог разглядеть, как из темноты медленно выплыла голова огромного паука


read more at ТСНrss2lj


Будь проклят день,


## Замеры памяти

In [24]:
b_size = 2
random.shuffle(train_dataset)
batch = data_collator(train_dataset[:b_size])
optim = torch.optim.Adam(model.parameters(),lr=1e-5)

prof = Profiler()
prof.one_step_report(batch, model,optim,device = DEVICE)

🤖 begin gpu mem : 12457.0/15102.1 mb
🤖 0.135s forward gpu mem : 12457.0/15102.1 mb
🤖 0.208s backward gpu mem : 12457.0/15102.1 mb
🤖 0.045s optimizer_step gpu mem : 12457.0/15102.1 mb


,used_mem,delta_mem,delta_time
begin,12457.0000,0.0,0.000
forward,12457.0000,0.0,0.135
backward,12457.0000,0.0,0.208
optim_step,12457.0000,0.0,0.045
end,4327.0000,-8130.0,0.090
total,15102.0625,0.0,0.478


## Useful links

1. [Tokenizers tutorial](https://huggingface.co/docs/transformers/tokenizer_summary) - brief analysis of all types of tokenizers from Huggingface with examples.
1. [How to generate text](https://huggingface.co/blog/how-to-generate) - overview of how to sample text using language models (bimsurch, etc.).
1. [Attention is All You Need](https://arxiv.org/pdf/1706.03762.pdf) - original article about the first Transformer.
1. [GPT-1](https://openai.com/blog/language-unsupervised/) - an article on OpenAI blog about GPT-1.
1. [GPT-2](https://openai.com/blog/better-language-models/) - OpenAI blog article about GPT-2.
1. [GPT-3](https://openai.com/blog/gpt-3-apps/) - OpenAI blog article about GPT-3.
1. [WebGPT](https://openai.com/blog/improving-factual-accuracy/) - OpenAI blog article about GPT-3, trained to google.
1. [Codex](https://openai.com/blog/openai-codex/) - OpenAI blog article about GPT-3 trained to write code.